# Composing techniques: Digital Dynamical Decoupling and Zero Noise Extrapolation

Noise in quantum computers can arise from a variety of sources, and sometimes applying multiple error mitigation techniques can be more beneficial than applying a single technique alone. 

Here we apply a combination of Digital Dynamical Decoupling (DDD) and Zero Noise Extrapolation (ZNE) to a randomized benchmarking (RB) task.

In [DDD](../guide/ddd.md), the input quantum circuit is modified by adding in gate sequences at regular intervals designed to reduce interaction between (i.e., decouple) the qubits from their environment. 

In [ZNE](../guide/zne.md), the expectation value of the observable of interest is computed at different noise levels, and subsequently the ideal expectation value is inferred by extrapolating the measured results to the zero-noise
limit. 
More information on the DDD and ZNE techniques can be found in the corresponding sections of the user guide (linked
above).

## Setup

We begin by importing the relevant modules and libraries required for the rest of this tutorial.

In [116]:
import cirq
import numpy as np
from mitiq.benchmarks import generate_rb_circuits
from mitiq import MeasurementResult, Observable, PauliString

## Task

We will demonstrate using DDD + ZNE on RB circuits, which are generated using Mitiq's built-in benchmarking circuit generation function, `generate_rb_circuits()`. 
More information on the RB protocol is available in the [Randomized Benchmarking section](https://qiskit.org/ecosystem/experiments/manuals/verification/randomized_benchmarking.html) of the [Qiskit Experiments Manual](https://qiskit.org/ecosystem/experiments/manuals). 
In this example we use a two-qubit RB circuit with a Clifford depth (number of Clifford groups) of 10.

In [117]:
# TODO Perhaps better to use something with more structure, e.g. GHZ
circuit = generate_rb_circuits(2, 10)[0]

## Noise model and executor

**Importantly**, since DDD is designed to mitigate time-correlated (non-Markovian) noise, we simulate systematic $R_z$ rotations and depolarising noise applied to each qubit after each time step. This corresponds to noise which is strongly time-correlated and, therefore, likely to be mitigated by DDD.

We use an [executor function](../guide/executors.md) to run the quantum circuit with the noise model applied.

In [118]:
def execute(
    circuit: cirq.Circuit, 
    rz_noise: float = 0.05,
    depolar_noise: float = 0.001
    ) -> MeasurementResult:
    """
    Execute a circuit with R_z dephasing noise of strength ``rz_noise`` and 
    depolarizing noise ``depolar_noise``
    """
    # Simulate systematic dephasing (coherent RZ) on each qubit for each moment.
    circuit = circuit.with_noise(cirq.rz(rz_noise))

    # Simulate systematic depolarizing on each qubit for each moment.
    circuit = circuit.with_noise(cirq.bit_flip(depolar_noise))

    simulator = cirq.DensityMatrixSimulator()

    result = simulator.run(circuit, repetitions=1000)
    
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings)

## Observable

In this example, the observable of interest is $ZI + IZ$.

In [119]:
obs = Observable(PauliString("ZI"), PauliString("IZ"))

For the circuit defined above, the ideal (noiseless) expectation value of the $ZI + IZ$ observable is 2, but as we will see, the unmitigated (noisy) result is impacted by depolarizing and readout errors.

In [120]:
from functools import partial

ideal_exec = partial(execute, rz_noise = 0.0, depolar_noise = 0.0)

ideal = obs.expectation(circuit, ideal_exec)
print("Ideal value:", "{:.5f}".format(ideal.real))

Ideal value: 2.00000


In [121]:
noisy_exec = partial(execute, rz_noise = 0.0, depolar_noise = 0.0005)
noisy = obs.expectation(circuit, noisy_exec)
print("Unmitigated noisy value:", "{:.5f}".format(noisy.real))

Unmitigated noisy value: 1.85000


Next we choose our gate sequences to be used in the digital dynamical decoupling routine (DDD). 
More information on choosing appropriate sequences can be found in the [DDD theory](../guide/ddd-5-theory.md#common-examples-of-ddd-sequences) section of the user guide.

In [122]:
from mitiq import ddd

rule = ddd.rules.yy

# Pretty sure I need something like batched_executor or serial executor. 
#
ddd_executor = ddd.mitigate_executor(noisy_exec, observable=obs, rule=rule)

ddd_result = ddd_executor(circuit)
print("Mitigated value obtained with DDD:", "{:.5f}".format(ddd_result.real))

Mitigated value obtained with DDD: 1.88400


We can see that REM improves the results, but errors remain.
For comparison, we then apply ZNE without DDD.

In [123]:
from mitiq import zne

zne_executor = zne.mitigate_executor(noisy_exec, observable=obs, scale_noise=zne.scaling.folding.fold_global)
zne_result = zne_executor(circuit)
print("Mitigated value obtained with ZNE:", "{:.5f}".format(zne_result.real))

Mitigated value obtained with ZNE: 2.11000


Finally, we apply a combination of DDD and ZNE.
DDD is applied first to apply the control pulses to each circuit which ZNE runs to do its extrapolation.

In [129]:
combined_executor = zne.mitigate_executor(ddd_executor, scale_noise=zne.scaling.folding.fold_global)

combined_result = combined_executor(circuit)
print("Mitigated value obtained with DDD + ZNE:", "{:.5f}".format(combined_result.real))

ValueError: Could not parse executed results from executor with type typing.Union[float, typing.Tuple[float, typing.Dict[str, typing.Any]]].

From this example we can see that each technique affords some improvement, and for this specific noise model, the combination of DDD and ZNE is more effective in mitigating errors than either technique alone.